# Bio Benchmark
This notebook will benchmark some systems relevant to biological applications using exact-exchange.

In [ ]:
from os.path import join
import ipywidgets as widgets
w = widgets.Dropdown(options=["1uao", "1crn", "1oxr", "1a94", "1gfl"],
                     description="System:")
display(w)

In [ ]:
geom = w.value
sendir = "input"

In [ ]:
%load_ext remotemanager

In [ ]:
from remotecomputer.spring import SpringGPU
url = SpringGPU()
url.conda = "remote"
remote_dir = join("Runs", "2023", "hybrid")
url.mpi = 1
url.omp = 32
url.queue = "autumn4"

Now try running it.

In [ ]:
%%execute_remotely url=url remote_dir=remote_dir extra_files_send=sendir : sname=geom
from BigDFT.IO import read_pdb
from BigDFT.Inputfiles import Inputfile
from BigDFT.Calculators import SystemCalculator
from os.path import join

# System
with open(join("input", sname + ".pdb")) as ifile:
    sys = read_pdb(ifile)

# Input
inp = Inputfile()
inp.set_xc("PBE0")
inp.set_hgrid(0.5)
inp.set_psp_nlcc()
inp["psolver"] = {"setup": {"accel": "CUDA"}}

# Compute
calc = SystemCalculator(skip=True)

# Determine the run name
log = calc.run(sys=sys, input=inp, name=sname)